In [1]:
import pandas as pd

## CSV读取

In [ ]:
ratings = pd.read_csv('../data/ml-25ms/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [3]:
user_item_counts = ratings.groupby('userId')['movieId'].nunique().reset_index()
user_item_counts.columns = ['userId', 'item_count']
user_item_counts['item_count'].min(), user_item_counts['item_count'].max()

(20, 32202)

In [4]:
ratings['rating'].min(), ratings['rating'].max()

(0.5, 5.0)

In [5]:
movies = pd.read_csv('../data/ml-25m/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
item_entities = movies.iloc[:, :2].values.tolist()  # 只取前两列
item_entities = [[str(x), str(y)] for x, y in item_entities]
item_entities[:10]

[['1', 'Toy Story (1995)'],
 ['2', 'Jumanji (1995)'],
 ['3', 'Grumpier Old Men (1995)'],
 ['4', 'Waiting to Exhale (1995)'],
 ['5', 'Father of the Bride Part II (1995)'],
 ['6', 'Heat (1995)'],
 ['7', 'Sabrina (1995)'],
 ['8', 'Tom and Huck (1995)'],
 ['9', 'Sudden Death (1995)'],
 ['10', 'GoldenEye (1995)']]

## dat读取

In [19]:
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])
ratings.head()

/tmp/ipykernel_3400/874908152.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [20]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   userId     1000209 non-null  int64
 1   movieId    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [29]:
# 计算每个用户的评分次数
user_rating_counts = ratings.groupby('movieId').size().reset_index(name='rating_count')
# 找出最小评分次数
min_rating_count = user_rating_counts['rating_count'].min()
print(f"用户评分的最小次数: {min_rating_count}")
print(f"用户评分的最小次数: {user_rating_counts['rating_count'].max()}")

用户评分的最小次数: 1
用户评分的最小次数: 3428


In [3]:
ratings['rating'].max(), ratings['rating'].min()

(5, 1)

In [4]:
ratings['userId'].max(), ratings['userId'].min()

(6040, 1)

In [10]:
movies = pd.read_csv('../data/ml-10M100K/movies.dat', sep='::', names=['movieId', 'title', 'genres'])
movies.head()

C:\Users\dell\AppData\Local\Temp\ipykernel_3684\1545558289.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('../data/ml-10M100K/movies.dat', sep='::', names=['movieId', 'title', 'genres'])


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
ratings[ratings['rating'] > 2]

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [12]:
from collections import Counter

df = ratings.sort_values(['userId', 'timestamp'])  # sort for user and time

# 删除评分小于 score 的交互
df = df[df['rating'] > 2]

# 获取用户物品交互列表
user_item = df[['userId', 'movieId']].values.tolist()
user_item = [list(map(str, line)) for line in user_item]

users, items = zip(*user_item)
user_list = []
item_list = []

users_cnt = Counter(users)
for user, count in users_cnt.items():
    if count >= 10:
        user_list.append(user)

items_cnt = Counter(items)
for item, count in items_cnt.items():
    if count >= 10:
        item_list.append(item)

num_user, num_item = len(user_list), len(item_list)

In [28]:
len(user_item), len(item_list)

(836478, 3123)

In [15]:
user2id, item2id = reverse_and_map(user_list), reverse_and_map(item_list)
len(user2id), len(item2id)

(6033, 3123)

In [27]:
min(item2id.keys()), max(item2id.keys())

('1', '999')

In [25]:
res = []
for x, y in user_item:
    if x in user2id.keys() and y in item2id.keys():
        res.append([user2id[x], item2id[y]])
    else:
        print(x, y)
        print(x in user2id.keys())
        print(y in item2id.keys())
        break

5 1715
True
False


In [23]:
len(res)

834449